# Redes neuronals

Ahora querremos aprender a clasificar imágenes, a partir de un entrenamiento con imágenes y etiquetas (aprendizaje supervisado). Con esta intención, utilizaremos el módulo de Keras (que utiliza por debajo de TensorFlow de Google). Para poder ver las imágenes, utilizaremos matplotlib.

In [ ]:
# Copy datasets, no usar en local, solo para MareNostrum
! cp ./datasets/mnist.npz ~/.keras/datasets/
! cp ./datasets/cifar-10-python.tar.gz ~/.keras/datasets/cifar-10-batches-py.tar.gz

In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import copy
import tensorflow

## MNIST
En el primer experimento utilizaremos el dataset MNIST. MNIST es un dataset de imágenes de números escritos a mano. El objetivo de la tarea es aprender a reconocer los 10 dígitos.

In [ ]:
#Import dataset from Keras
from tensorflow.python.keras.datasets import mnist
#Load train data, train labels, test data and test labels.
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print( "Nombre d'imatges per entrenament:", x_train.shape[0])
print( "Nombre d'imatges per avaluació:", x_test.shape[0])
print( "Tamany de les imatges:", x_train.shape[1:])

<img src=img/mnist.png width='450px'>

[Image source](https://www.researchgate.net/publication/306056875_An_analysis_of_image_storage_systems_for_scalable_training_of_deep_neural_networks)

### Preprocesado

Tenemos 60.000 imágenes de 28x28 píxeles. Si fueran imágenes en color, necesitaríamos otra dimensión para representar los 3 canales del color (es decir, 28x28x3, usando los colores primarios rojo, verde y azul o RGB).
Visualizamos una de las imágenes del dataset:

In [ ]:
# Elige una imagen del conjunto de entrenamiento al azar
import random
index_imatge = random.randint(1,x_test.shape[0])
# Visualización utilizando matplotlib
plt.title('Etiqueta es {label}'.format(label=y_train[index_imatge]))
plt.imshow(x_train[index_imatge], cmap='gray')
plt.show()

Para introducir las imágenes en una FNN necesitamos prepararlas. Consideraremos cada pixel de la imagen como una variable independiente, así que convertiremos la imagen en un vector de longitud 784 (28x28). Cada neurona de la primera capa de la FNN tendrá una entrada de 784 valores.

<center><img src=img/mnist_reshape.png width='800px'></center>

[Image source](https://puture.tistory.com/385)

In [ ]:
#Transforma les imatges a vectors
x_train = x_train.reshape(60000, 28*28)
#Fem una copia per us posterior
original_test = copy.deepcopy(x_test)
x_test = x_test.reshape(10000, 28*28)

También se recomienda normalizar los datos. En estos momentos, los valores del vector son números naturales entre 0 (negro) y 255 (blanco). La red aprenderá más fácilmente si normalizamos estos valores en el rango [0,1] usando decimales.

In [ ]:
print("Fragment del vector pre-normalització:", x_train[0, 201:206])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255
print("Fragment del vector post-normalització:", x_train[0, 201:206])

También debemos ajustar las etiquetas asociadas a las imágenes a las necesidades de la FNN. La etiqueta en este caso es un dato 'categórico', es decir: puede tomar un valor de un conjunto de valores finito en los que no existe orden. Aunque en este problema parece haber un orden entre las etiquetas, este orden no es relevante de cara a clasificar imágenes.

La salida de la red neuronal representa un conjunto de probabilidades de cada imagen. Por ejemplo, la salida:
[0, 0, 0, 0, 0.25, 0, 0.5, 0.25, 0, 0]
Indica una probabilidad del 25% de las etiquetas '4' y '7' y una probabilidad del 50% de la etiqueta '6'.
Para poder calcular el error de la red en su predicción, utilizamos una representación análoga de las etiquetas. Así pues una imagen con etiqueta '6' tendrá la siguiente representación:
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Y el error de la red se calculará como la diferencia entre ambos vectores. Este tipo de representación se llama one-hot vector encoding.

<img src=img/one_hot.PNG width='500px'>

In [ ]:
from tensorflow.python.keras.utils import to_categorical
print("Format original:",y_train[0])
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
print("Format one-hot:",y_train[0])

### Red neuronal Feed-forward

Ahora, ya podremos pasar a definir nuestra red. Empecemos con una red neuronal de dos capas con 32 y 16 neuronas respectivamente. Todas con función de activación ReLU. La capa de salida tendrá 10 neuronas (1 por clase/etiqueta), usando la función softmax para obtener probabilidades por clase.

In [ ]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense

nn = Sequential()
nn.add(Dense(32,activation='relu',input_shape=(784,)))
nn.add(Dense(16,activation='relu'))
nn.add(Dense(10, activation='softmax'))

Ahora nos queda definir el optimizador y sus parámetros. Optimizaremos utilizando 'Stochastic Gradient Descent' (sgd), evaluando nuestro error con la función 'Categorical Crossentropy' (que es la adecuada cuando utilizamos softmax), y optimizaremos para mejorar la 'accuracy' del modelo.

In [ ]:
nn.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

<img src=img/categorical_cross_entropy.png width='400px'>
<img src=img/loss.png width='400px'>


Fuente de las imágenes y más información sobre la 'cross-entropy' [aquí](https://towardsdatascience.com/cross-entropy-for-classification-d98e7f974451).

Ya podemos empezar a entrenar la red con los datos de entrenamiento. Lo haremos en lotes de 128 imágenes (batch size). Cuando hayamos visto todas las imágenes (1 epoch), volveremos a empezar. En total, pasaremos cada imagen 10 veces por la red (10 epochs).

En el output vemos en qué epoch estamos, el error cometido en esa epoch (loss) que la red pretende minimizar, y la precisión obtenida (accuracy) en los datos de entrenamiento.

In [ ]:
#Start training
nepochs=10
history = nn.fit(x_train,y_train,batch_size=128,epochs=nepochs)

El rendimiento en el conjunto de entrenamiento es suficientemente bueno (91%-92% de precisión). Pero falta por ver si éste se mantiene en los datos de evaluación. Así veremos si el modelo entrenado generaliza bien a nuevos datos.

In [ ]:
score = nn.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

Aunque los números no son los mismos, parece que la red generaliza bastante bien.
Ahora vamos a visualizar el entrenamiento con las curvas de precisión y error.

In [ ]:
acc_plot = plt.figure(1)
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.xticks([x for x in range(nepochs)], [x+1 for x in range(nepochs)])
plt.legend(['train'], loc='upper left')
plt.show()

loss_plot = plt.figure(2)
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xticks([x for x in range(nepochs)], [x+1 for x in range(nepochs)])
plt.legend(['train'], loc='upper left')
plt.show()

Parece que la red aprende de forma continuada. También parece que la tendencia era a mejorar, así que quizás sería buena idea darle más epochs para obtener mejores resultados.

Por último, vamos a ver la relación de aciertos y errores, usando una matriz de confusión. La matriz de confusión muestra las predicciones hechas junto a las etiquetas reales. Permite ver dónde se han cometido los errores.

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
import seaborn as sn
import pandas as pd

#Predir el test i contrastar-lo amb els valors reals.
Y_pred = nn.predict(x_test)
y_pred = np.argmax(Y_pred, axis=1)

df_cm = pd.DataFrame(confusion_matrix(np.argmax(y_test,axis=1), y_pred), index = range(10), columns = range(10))
plt.figure(figsize = (10,7))
ax = sn.heatmap(df_cm, annot=True, annot_kws={"size": 8}, cmap="YlGnBu", fmt='g');
plt.ylabel("Etiqueta real")
plt.xlabel("Etiqueta predita")
plt.show()

Los errores parecen estar muy repartidos. La etiqueta peor predicho es el 5, que muy a menudo se clasifica como la etiqueta 3.

Vamos a ver ejemplos de errores cometidos. Muchos son errores razonables.

In [ ]:
errors = [(i, predit, np.argmax(real)) for i,(predit, real) in enumerate(zip(y_pred, y_test)) if predit!=np.argmax(real)]

In [ ]:
import random
index_error = random.randint(0,len(errors))
error_a_analitzar = errors[index_error]
plt.title("Predit:"+str(error_a_analitzar[1])+"; Real:"+str(error_a_analitzar[2]))
plt.imshow(original_test[error_a_analitzar[0]], cmap='gray')
plt.show()

Como puede verse, los errores son las imágenes especialmente problemáticas.

## CIFAR DATASET

Ahora vamos a trabajar con un problema algo más complicado. Imágenes en color representando varios objetos.

In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import copy
from tensorflow.python.keras import utils

Importamos el dataset CIFAR10, que tiene clases de animales y vehículos.

In [ ]:
from tensorflow.python.keras.datasets import cifar10

(original_x_train, y_train), (original_x_test, y_test) = cifar10.load_data()
# Located in ~/.keras/datasets and probably need untarred
x_train = copy.deepcopy(original_x_train)
x_test = copy.deepcopy(original_x_test)
print( "Número d'imatges per entrenament:", x_train.shape[0])
print( "Número d'imatges per avaluació:", x_test.shape[0])
print( "Tamany de les imatges:", x_train.shape[1:])

En este caso, las imágenes tienen profundidad, ya que tenemos canal de color. Habrá que adaptar el código.

Para entender mejor las clases, https://www.cs.toronto.edu/~kriz/cifar.html tiene una guía que nos transforma la etiqueta numérica en la clase en palabras.

In [ ]:
label_map=['airplane','automobile', 'bird', 'cat','deer','dog','frog','horse','ship','truck']
import random
index_imatge = random.randint(0,9)
plt.title("L'etiqueta és {label}".format(label=label_map[y_train[index_imatge][0]]))
plt.imshow(x_train[index_imatge], cmap='gray')
plt.show()

Prepararemos los datos al igual que antes. Ahora, los vectores serán de 3,072 posiciones (32x32x3).

In [ ]:
x_train = x_train.reshape(50000, 32*32*3)
x_test = x_test.reshape(10000, 32*32*3)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense

nn = Sequential()
nn.add(Dense(32, activation='relu', input_shape=(32*32*3,)))
nn.add(Dense(16, activation='relu'))
nn.add(Dense(10, activation='softmax'))
nn.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
nepochs=20
history = nn.fit(x_train,y_train,batch_size=128,epochs=nepochs)

In [ ]:
acc_plot = plt.figure(1)
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.xticks([x for x in range(nepochs)], [x+1 for x in range(nepochs)], fontsize=10)
plt.yticks(fontsize=10)
plt.legend(['train'], loc='upper left')
plt.show()

loss_plot = plt.figure(2)
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xticks([x for x in range(nepochs)], [x+1 for x in range(nepochs)], fontsize=10)
plt.yticks(fontsize=10)
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
score = nn.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

Los resultados no son tan buenos en este problema. Alrededor del 40%.
¿Puede extraer cuáles son las clases del test donde más se equivoca?
¿Puede ver las imágenes erróneas?
¿Puede mejorar la accuracy de test?
¿Y cuánto la puede mejorar?

# Tornem a les slides!
<img src="img/end_1.png">

### Redes convolucionales

El preprocessing es similar al anterior, pero las neuronas convolucionales de la primera capa necesitan las entradas en 3 dimensiones. El orden de las dimensiones depende del backend.

In [ ]:
from tensorflow.python.keras import backend as K

img_rows, img_cols, channels = 32, 32, 3
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], channels, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], channels, img_rows, img_cols)
    input_shape = (channels, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)
    input_shape = (img_rows, img_cols, channels)
print(x_train.shape)

Definimos nuestra red. Volveremos a tener tres capas ocultas de 32, 16 y 16 neuronas, y una de salida que será una softmax de 10 posiciones.

A diferencia de que antes, sin embargo, las dos primeras capas serán convolucionales. Ambas usarán kernels (filtros) de 2 por 2. Después de cada capa convolucional, haremos un 'pooling', cogiendo el valor máximo de cada cuatro píxeles (para reducir las dimensiones de la salida de la capa). Por último, entre la segunda capa convolucional y la primera capa no convolucional (fully connected), deberemos aplanar los datos a un vector (como antes hemos hecho en el preprocessing).

Volveremos a utilizar los mismos parámetros por optimización que antes.

In [ ]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense 

nn = Sequential()
nn.add(Conv2D(filters=32, kernel_size=(2, 2), input_shape=input_shape, activation='relu', padding='same', strides=(2,2)))
nn.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
nn.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=16))
nn.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
nn.add(Flatten())
nn.add(Dense(16, activation='relu'))
nn.add(Dense(10, activation='softmax'))
nn.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
#print(nn.summary())

Notemos que ahora, al entrenar, nos dejamos un 15% de imágenes por validación. Por tanto, también nos aparecerán la loss y la accuracy de la validación aparte. El entrenamiento de la nueva red tardará algo más que antes.

In [ ]:
nepochs=10
history = nn.fit(x_train,y_train,batch_size=128, epochs=nepochs, validation_split=0.2)

De nuevo, realizaremos un estudio sobre nuestros propios resultados

In [ ]:
acc_plot = plt.figure(1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.xticks([x for x in range(nepochs)], [x+1 for x in range(nepochs)])
plt.legend(['train','val'], loc='upper left')
plt.show()

loss_plot = plt.figure(2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xticks([x for x in range(nepochs)], [x+1 for x in range(nepochs)])
plt.legend(['train','val'], loc='upper left')
plt.show()


score = nn.evaluate(x_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

¿Cómo mejoraría las redes? ¿Cuál es el mejor resultado de accuracy en el test que puede obtener?